In [ ]:
import pandas as pd
import altair as alt
import re

In [ ]:
bed_overlap = '/Users/anton/sandbox/vgp/zebraFinch/overlaps.txt'
parsed = '/Users/anton/sandbox/vgp/zebraFinch/ovrlp_parsed.txt'
all_orfs = '/Users/anton/sandbox/vgp/zebraFinch/orfs.bed'

In [ ]:
ovlp = {}

orf_id = re.compile('^ID=([\w\_\.\d]*);')
with open(bed_overlap,'r') as f:
    for line in f:
        fields = line.rstrip().split('\t')
        id1 = orf_id.match(fields[3]).group(1)
        id2 = orf_id.match(fields[9]).group(1)
        if id1 != id2:
            ovrp_id = ''.join(sorted([id1,id2]))
            ovlp[ovrp_id] = [id1,id2,int(fields[len(fields)-1])]

In [ ]:
with open(parsed,'w') as f:
    for key in ovlp.keys():
        f.write('{},{},{}\n'.format(ovlp[key][0],ovlp[key][1],ovlp[key][2]))

In [ ]:
del(ovlp)

In [ ]:
overlap = pd.read_csv(parsed,names=['orf1','orf2','l'])

In [ ]:
orf_data = {
    'id':[],
    'start':[],
    'end':[],
    'type':[],
    'len':[],
    'frame':[],
    's_codon':[],
    'e_codon':[]
}
with open(all_orfs,'r') as f:
    for line in f:
        line = line.rstrip()
        fields = line.split('\t')
        data = fields[3].split(';')
        orf_data['start'].append(int(fields[1]))
        orf_data['end'].append(int(fields[2]))
        orf_data['id'].append(data[0][3:])
        orf_data['type'].append(data[1][9:])
        orf_data['len'].append(int(data[2][8:]))
        orf_data['frame'].append(int(data[3][10:]))
        orf_data['s_codon'].append(data[4][6:])
        orf_data['e_codon'].append(data[5][5:])        

In [ ]:
data = pd.DataFrame.from_dict(orf_data)

In [ ]:
del(orf_data)

In [ ]:
overlap = overlap.merge(data,left_on='orf1',right_on='id',how='left')

In [ ]:
overlap.rename(columns={"start":"s1","end":"e1"},inplace=True)

In [ ]:
overlap = overlap.merge(data,left_on='orf2',right_on='id',how='left')

In [ ]:
overlap.rename(columns={"start":"s2","end":"e2"},inplace=True)

In [ ]:
overlap.head()

In [ ]:
overlap=overlap['orf1,orf2,l,s1,e1,s2,e2,len_x,frame_x,s_codon_x,e_codon_x,len_y,frame_y,s_codon_y,e_codon_y'.split(',')]

In [ ]:
overlap.head()

In [ ]:
# Nested case
overlap[
    (
    ( ( overlap['s1'] <= overlap['s2'] ) & ( overlap['e1'] >= overlap['e2'] ) ) |
    ( ( overlap['s1'] >= overlap['s2'] ) & ( overlap['e1'] <= overlap['e2'] ) )
    )
]

In [ ]:
# Nested case
overlap[
    (
    ( ( overlap['s1'] <= overlap['s2'] ) & ( overlap['e1'] <= overlap['e2'] ) ) 
    )
]